In [1]:
from transformers import AutoModel, AutoTokenizer
import torch
import torch.nn as nn
import csv
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
import sys
import random
import pandas as pd
from rerank_score_cands_new import load_cands
import numpy as np
from comet import download_model, load_from_checkpoint
import pickle
from sklearn.utils import shuffle
#from distill_comet import XLMCometRegressor

csv.field_size_limit(sys.maxsize)
xlm_tok = AutoTokenizer.from_pretrained('xlm-roberta-base')


2022-10-03 10:58:50.034370: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-03 10:58:50.034393: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
class XLMCometRegressor(nn.Module):
    
    def __init__(self, drop_rate=0.1):
        # TODO should we be freezing layers?
        super().__init__()
        
        self.xlmroberta = AutoModel.from_pretrained('xlm-roberta-base')
        # Num labels 1 should just indicate regression (?)
        self.regressor = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(self.xlmroberta.config.hidden_size, 1), 
        )
        self.to(device)
        
    def forward(self, input_ids, attention_masks):
        # don't finetune xlmroberta model
        #with torch.no_grad():
        word_rep, sentence_rep = self.xlmroberta(input_ids, attention_mask=attention_masks, encoder_attention_mask=attention_masks, return_dict=False)
        # use the first <s> token as a CLS token, TODO experiment with using the sum of 
        # ensure padding not factored in
        word_rep = word_rep*(input_ids>0).unsqueeze(-1)
        outputs = self.regressor(torch.sum(word_rep, 1))
        #print("Shape: ", outputs.shape)
        return outputs

In [3]:
# nothing is everything
# 1 is with distilled data lattice + beam
# 2 is with just beam data? 
# ordered is the clean ranking ready set
def load_cometqe_data(ind):
    with open('processeddata/orderedmasks.pkl', 'rb') as f:
        masks = pickle.load(f)

    with open('processeddata/orderedinps.pkl', 'rb') as f:
        xinps = pickle.load(f)

    with open('processeddata/orderedlabels.pkl', 'rb') as f:
        yinps = pickle.load(f)
    return masks, xinps, yinps

mdata, xdata, ydata = load_cometqe_data(1)

In [4]:
# train test split
cut = int(len(xdata)*.9)- int(len(xdata)*.9)%32
xtrain, ytrain, mtrain = xdata[:cut], ydata[:cut], mdata[:cut]
xtest, ytest, mtest = xdata[cut:], ydata[cut:], mdata[cut:]

In [5]:
del mdata, xdata, ydata
torch.cuda.empty_cache()

In [6]:
testset = torch.tensor(ytrain[:31]+[.8]).to(device)

In [7]:
ytrain[:32]

[0.5608487725257874,
 0.6321543455123901,
 0.6690516471862793,
 0.66994708776474,
 0.7340843081474304,
 0.7379187941551208,
 0.7536868453025818,
 0.759560227394104,
 0.7654272317886353,
 0.7689279913902283,
 0.7720431685447693,
 0.7778377532958984,
 0.7796866893768309,
 0.7859623432159424,
 0.7919414043426514,
 0.7928602695465088,
 0.7943693399429321,
 0.8000797033309937,
 0.8007832169532776,
 0.8013631701469421,
 0.8021320104598999,
 0.8032323718070984,
 0.8044326901435852,
 0.8044593930244446,
 0.8061469197273254,
 0.8118272423744202,
 0.8131868839263916,
 0.8135043978691101,
 0.817229688167572,
 0.8339771032333374,
 0.8789095878601074,
 0.887186586856842]

In [8]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

class RegressionDataset(Dataset):
    def __init__(self, sentences, labels, masks):
        assert len(sentences) == len(labels)
        self.sentences = sentences
        self.labels = labels
        self.masks = masks

    def __getitem__(self, i):
        return self.sentences[i], self.labels[i], self.masks[i]

    def __len__(self):
        return len(self.sentences)

def collate_custom(datafull):
    #print(len(datafull[0]))
    data = [torch.tensor(d[0]) for d in datafull]
    masdata=  [d[2] for d in datafull]
    labels = [d[1] for d in datafull]
    max_len = max([x.squeeze().numel() for x in data])
    data = [torch.nn.functional.pad(x, pad=(0, max_len - x.numel()), mode='constant', value=0) for x in data]
    data = torch.stack(data).to(device)
    # TODO just a normal mask for now
    #masdata = [torch.ones_like(m) for m in masdata]
    masdata = [torch.nn.functional.pad(x, pad=(0, max_len - x[0].numel(), 0, max_len - x[0].numel()), mode='constant', value=0) for x in masdata]
    masdata = torch.stack(masdata).to(device)
    return data, torch.tensor(labels).to(device), masdata

testloader = DataLoader(RegressionDataset(xtest, ytest, mtest), batch_size=32, shuffle=False, collate_fn=collate_custom)
trainloader = DataLoader(RegressionDataset(xtrain, ytrain, mtrain), batch_size=32, shuffle=False, collate_fn=collate_custom)

In [ ]:
tinmax = 100000
xtiny, ytiny, mtiny = xtrain[:tinmax], ytrain[:tinmax], mtrain[:tinmax]
tinyloader = DataLoader(RegressionDataset(xtiny, ytiny, mtiny), batch_size=32, shuffle=True, collate_fn=collate_custom)

In [ ]:
it = iter(testloader)

In [9]:
# hyperparams
from transformers import AdamW, get_linear_schedule_with_warmup

In [10]:
from torch.nn.utils.clip_grad import clip_grad_norm_
def train(model, optimizer, scheduler, loss_function, epochs,       
          train_dataloader, device, clip_value=2):
    print("Total steps :", epochs*len(train_dataloader))
    best_loss = 1e10
    for epoch in range(epochs):
        if epoch%1==0:
            print("EPOCH ", epoch)
            print("-----")
            print(best_loss)
        model.train()
        cbest = 1e10
        lostot = 0
        loscnt = 0
        for step, batch in enumerate(train_dataloader): 
            batch_inputs, batch_labels, batch_masks = \
                               tuple(b.to(device) for b in batch)
            model.zero_grad()
            outputs = model(batch_inputs, batch_masks)
            loss = loss_function(outputs.squeeze(), 
                             batch_labels.squeeze())
            lostot+=loss
            loscnt+=1
            if step%500==0:
                #print(loss)  
                if loscnt>0:
                    print(lostot/loscnt)
                    cbest = min(float(lostot/loscnt), cbest)
                    best_loss = min(best_loss, cbest)
                    print("cbest, ", cbest)
            loss.backward()
            clip_grad_norm_(model.parameters(), clip_value)
            optimizer.step()
            scheduler.step()
        cbest = min(float(lostot/loscnt), cbest)
        best_loss = min(best_loss, cbest)
        print("cbest, ", cbest)
        if cbest==best_loss:
            torch.save(model.state_dict(), "torchsaved/maskedminicomestim"+str(epoch)+".pt")
    return model

def evaluate(model, loss_function, tdataloader, device):
    model.eval()
    test_loss, test_r2 = [], []
    preds = []
    ind = 0
    for batch in tdataloader:
        batch_inputs, batch_labels,batch_masks = \
                                 tuple(b.to(device) for b in batch)
        with torch.no_grad():
            outputs = model(batch_inputs, batch_masks)
        loss = loss_function(outputs.squeeze(), 
                             batch_labels.squeeze())
        preds.append(list(outputs.squeeze()))
        test_loss.append(loss.item())
        #r2 = r2_score(outputs, batch_labels)
        #test_r2.append(r2.item())
        if ind==10:
            print(batch_labels)
            print(outputs)
        ind+=1
    return test_loss, preds

In [11]:
del model

NameError: name 'model' is not defined

In [12]:
model = XLMCometRegressor(drop_rate=0.1)
model.load_state_dict(torch.load("./torchsaved/maskedcont3.pt"))

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [20]:
# maskedcont1 has first round of optimizing loss
# maskedcont2 more contrastive learning
# maskedcont3 full on contrastive 
torch.save(model.state_dict(), "torchsaved/maskedcont4.pt")

In [13]:
torch.cuda.empty_cache()

In [16]:
vmask = (torch.triu(torch.ones(32, 32))*2-torch.ones(32, 32))*-1
vmask = vmask.to(device)
mse = nn.MSELoss()
def rank_loss(preds, golds):
    totloss = 0
    for i in range(1, len(preds)):
        # for margin
        margin = (golds - torch.roll(golds, i))*vmask[i]
        diff = (preds - torch.roll(preds, i)-margin)*vmask[i]
        diff[diff<0] = 0
        totloss+=torch.sum(diff)
    return totloss + mse(preds, golds)

In [17]:
def run_model_train_params(learn_r, epochs, loader, mod):
    optimizer = AdamW(mod.parameters(),
                      lr=learn_r,
                      eps=1e-8)
    scheduler = get_linear_schedule_with_warmup(optimizer,       
                     num_warmup_steps=0, num_training_steps=epochs*len(loader))
    model = train(mod, optimizer, scheduler, rank_loss, epochs, 
                  loader, device, clip_value=2)

In [19]:
run_model_train_params(1e-6, 40, trainloader, model)

Total steps : 98440
EPOCH  0
-----
10000000000.0
tensor(0.6183, device='cuda:1', grad_fn=<DivBackward0>)
cbest,  0.6183037757873535
tensor(1.1864, device='cuda:1', grad_fn=<DivBackward0>)
cbest,  0.6183037757873535
tensor(1.2136, device='cuda:1', grad_fn=<DivBackward0>)
cbest,  0.6183037757873535
tensor(1.1971, device='cuda:1', grad_fn=<DivBackward0>)
cbest,  0.6183037757873535
tensor(1.2129, device='cuda:1', grad_fn=<DivBackward0>)
cbest,  0.6183037757873535
cbest,  0.6183037757873535
EPOCH  1
-----
0.6183037757873535
tensor(0.7793, device='cuda:1', grad_fn=<DivBackward0>)
cbest,  0.77928227186203
tensor(1.0859, device='cuda:1', grad_fn=<DivBackward0>)
cbest,  0.77928227186203
tensor(1.0957, device='cuda:1', grad_fn=<DivBackward0>)
cbest,  0.77928227186203
tensor(1.1028, device='cuda:1', grad_fn=<DivBackward0>)
cbest,  0.77928227186203
tensor(1.1142, device='cuda:1', grad_fn=<DivBackward0>)
cbest,  0.77928227186203
cbest,  0.77928227186203
EPOCH  2
-----
0.6183037757873535
tensor(0.92

In [14]:
for t in trainloader:
    #print(t[1])
    assert t[1][-1] == max(t[1])

In [69]:
del model 
model = XLMCometRegressor(drop_rate=0.1)
model.load_state_dict(torch.load("./torchsaved/maskedcont2.pt"))

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [22]:
loss = evaluate(model, rank_loss, testloader, device)

In [53]:
import math
import matplotlib.pyplot as plt

In [ ]:
len(xtrain)

In [ ]:
sloss = [math.sqrt(l) for l in loss[0]]

In [ ]:
nval = [float(l) for l in loss[1]]

In [28]:
mind = []
for l in loss[1]:
    mind+=[l.index(max(l))]

sum(mind)/len(mind)

30.48905109489051

In [39]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(alldf['scores'])

In [ ]:
# old data loading stuff

# data comes from WMT 2019 [I think], TODO validate exact year being used
# data generated with gold reference given as hyp
#golddf = pd.read_csv('./processeddata/golddata.csv')
#golddf['inp'] = golddf['fr']
#golddf['hyp'] = golddf['en']
# data generated on candidates from beam search 50 and lattice (lots of bad)
distill_df = pd.read_csv('distill_cometdata_1.csv')
# data generated between random / unrelated sentences 
#rand_df = pd.read_csv('distill_cometdata_rand.csv')[:30000]
# combine
alldf = distill_df#.append(rand_df, ignore_index=True)

In [ ]:
sorted3 = alldf.sort_values(['ref', 'scores']).reset_index().drop(columns=['index', 'Unnamed: 0'])
sorted3 = sorted3[sorted3["inp"].str.contains("&#")==False]
sorted3 = sorted3[sorted3['inp'].str.len()>40]
sorted3 = sorted3[sorted3['ref'].str.len()>40]

In [ ]:
# go through dataframe, get sets of size BATCH that are ranked by score
dlists = []
prevs = sorted3['inp'].iloc[0]
sofar = []
for index, row in sorted3.iterrows():
    if row['inp']==prevs:
        sofar.append(row)
    else:
        dlists.append(sofar)
        sofar = []
        prevs = row['inp']

In [ ]:
import random

In [ ]:
def get_samples(dlist, best, batch):
    nbatches = int(len(dlist)/batch)+1
    res = []
    for i in range(nbatches):
        # get a random sorted sample, make sure best is in each batch
        indices = random.sample(range(len(dlist)), batch-1)
        res.append([dlist[i] for i in sorted(indices)]+[best])
    return res

def all_samples(dls, batch):
    done = []
    for d in dls:
        done.extend(get_samples(d[:-1], d[-1], batch))
    return done

asamps = all_samples(dlists, 32)

In [ ]:
random.shuffle(asamps)

In [ ]:
done = []
for a in asamps:
    done.extend(a)

In [ ]:
done = pd.DataFrame(done)

In [ ]:
done = done.reset_index().drop(columns= ['index'])

In [ ]:
done.iloc[31+32*9]

In [ ]:
# construct inputs from dataframe
def get_inputs(inpdf):
    xinp = []
    yinp = []
    maskinp = []
    
    for index, row in inpdf.iterrows():
        if index%1000==0:
            print(index)
        #print(row['c1'], row['c2'])
        # will need to make a custom mask (maybe) so that inputs from both languages are encoded separately
        toktmp = xlm_tok(row['inp']).input_ids
        lent = len(toktmp)
        hyptmp = xlm_tok(row['hyp']).input_ids
        toktmp.extend(hyptmp)
        mask = torch.ones(len(toktmp), len(toktmp))
        # should set upper left and bottom right quadrants to 1, mask other stuff
        # TODO make different types of masks. 
        #mask[:lent, :lent] = 1
        #mask[lent:, lent:] = 1
        # make causal encoder-decoder mask
        mask[:lent, lent:] = 0
        mask[lent:, lent:] = torch.tril(mask[lent:, lent:])
        xinp.append(toktmp)
        maskinp.append(mask)
        yinp.append(row['scores'])
    return xinp, yinp, maskinp

In [ ]:
xdata, ydata, mdata = get_inputs(done)

In [ ]:
for i in range(0, len(xdata)):
    if len(xdata[i])>500:
        del xdata[i]
        del ydata[i]
        del mdata[i]

In [ ]:
# save all as precaution
def save_cometqe_data(md, xd, yd):
    # save data into a pickle file
    with open('processeddata/orderedmasks.pkl', 'wb') as f:
        pickle.dump(md, f)

    with open('processeddata/orderedinps.pkl', 'wb') as f:
        pickle.dump(xd, f)

    with open('processeddata/orderedlabels.pkl', 'wb') as f:
        pickle.dump(yd, f)
        
#mdata, xdata, ydata = load_cometqe_data()
        
save_cometqe_data(mdata, xdata, ydata)